<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_11_11_Multi_turn_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to use Graphlit to perform multi-step summarization of documents.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [3]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [4]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [5]:
from typing import List, Optional

async def ingest_uri(uri: str):
    if graphlit.client is None:
        return;

    try:
        # Using synchronous mode, so the notebook waits for the content to be ingested
        response = await graphlit.client.ingest_uri(uri=uri, is_synchronous=True)

        return response.ingest_uri.id if response.ingest_uri is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def create_openai_specification(model: enums.OpenAIModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"OpenAI [{str(model)}]",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.OPEN_AI,
        openAI=input_types.OpenAIModelPropertiesInput(
            model=model,
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def revise_content(prompt: str, content_id: str, specification_id: str, conversation_id: Optional[str] = None):
    if graphlit.client is None:
        return None, None

    try:
        response = await graphlit.client.revise_content(prompt=prompt, content=input_types.EntityReferenceInput(id=content_id), id=conversation_id, specification=input_types.EntityReferenceInput(id=specification_id))

        message = response.revise_content.message.message if response.revise_content is not None and response.revise_content.message is not None else None
        conversation_id = response.revise_content.conversation.id if response.revise_content is not None and response.revise_content.conversation is not None else None

        return message, conversation_id
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None, None

async def delete_all_specifications():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_specifications(is_synchronous=True)

async def delete_all_conversations():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_conversations(is_synchronous=True)

async def delete_all_contents():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_contents(is_synchronous=True)


Execute Graphlit example

In [6]:
from IPython.display import display, Markdown, HTML
import time

# Remove any existing contents, conversations and specifications; only needed for notebook example
await delete_all_conversations()
await delete_all_specifications()
await delete_all_contents()

print('Deleted all contents, conversations and specifications.')

uri = "https://graphlitplatform.blob.core.windows.net/test/documents/UMNwriteup.pdf"

content_id = await ingest_uri(uri=uri)

if content_id is not None:
    print(f'Ingested content [{content_id}]:')

Deleted all contents, conversations and specifications.
Ingested content [f0815445-3ae0-4634-a53f-aaad74cfe6d8]:


Create specification, and start multi-turn summarization conversation.

In [7]:
    specification_id = await create_openai_specification(enums.OpenAIModels.O1_MINI_128K)

    if specification_id is not None:
        print(f'Created specification [{specification_id}].')

        prompt = "Summarize this for me."

        message, conversation_id = await revise_content(prompt, content_id, specification_id)

        if message is not None:
            display(Markdown(f'**Revision:**\n{message}'))
            print()


Created specification [646cb568-b5ff-414d-ad8e-579ac9ac7b9f].


**Revision:**
The patient presents with chest pain and dyspnea, indicating possible ischemic heart disease and renovascular hypertension.

Risk factors include a history of hypertension, peptic ulcer disease, surgical menopause, and a family history of early atherosclerotic cardiovascular disease.

Recommended diagnostic steps involve admitting the patient to telemetry for monitoring, initiating diuretics to manage volume overload and high blood pressure, and performing a ventriculogram with catheterization to assess cardiac function.

Long-term care plans should include cholesterol monitoring, implementing an exercise and weight loss program, adopting a low-fat diet, and considering cholesterol-lowering medications if necessary.

Additional considerations involve evaluating for fibrocystic breast disease and assessing the significance of aortic stenosis suggested by the presence of a murmur.

In [8]:
        prompt = "OK, not bad, but make it longer."

        message, conversation_id = await revise_content(prompt, content_id, specification_id, conversation_id)

        if message is not None:
            display(Markdown(f'**Revision:**\n{message}'))
            print()


**Revision:**
The patient presents with chest pain and dyspnea, suggesting possible ischemic heart disease and renovascular hypertension. A detailed cardiac examination is essential to evaluate the severity and underlying causes of these symptoms.

Comprehensive assessment of all organ systems reveals no cyanosis, clubbing, or edema. Peripheral pulses in the femoral, popliteal, anterior tibial, dorsalis pedis, brachial, and radial areas are normal, indicating adequate peripheral circulation.

Vital signs should always be listed and monitored closely. Orthostatic blood pressure changes should be checked if relevant to the patient's complaints to rule out any autonomic dysfunction or volume depletion.

The abdominal examination shows a symmetrical abdomen without distention, normal bowel sounds, and no masses or splenomegaly. A bruit in the right paraumbilical area may suggest renal artery stenosis or other vascular abnormalities.

Neurological assessment includes checking for lumbo-sacral back pain, which the patient reports. This symptom should be evaluated to rule out any spinal or musculoskeletal issues contributing to the patient's overall condition.

Genital and rectal examinations are part of the comprehensive assessment to identify any unrelated or contributing factors to the patient's primary complaints. No abnormalities were noted in these areas.

The presence of a systolic murmur, particularly one that radiates to the neck, raises the possibility of aortic stenosis. However, a normal carotid upstroke may indicate that the murmur is not clinically significant.

An extensive diagnostic and therapeutic plan is necessary, incorporating both acute management and long-term care. This includes pharmacologic treatments like diuretics and platelet inhibitors, as well as non-pharmacologic measures such as a low-fat diet and exercise.

Appropriate laboratory work should include BUN/Creatinine to assess kidney function, electrolytes, and a baseline EKG to monitor cardiac activity. These tests are crucial for guiding treatment decisions and managing potential complications.

Long-term care should focus on cholesterol monitoring and implementing lifestyle changes, including an exercise and weight loss program. If cholesterol levels are elevated, cholesterol-lowering medications such as HMG-CoA reductase inhibitors may be necessary.

In [9]:
        prompt = "Rewrite from the perspective of a medical content writer. Add additional medical details. Format nicely with Markdown for a blog post."

        message, conversation_id = await revise_content(prompt, content_id, specification_id, conversation_id)

        if message is not None:
            display(Markdown(f'**Revision:**\n{message}'))
            print()


**Revision:**
# Comprehensive Management of a Patient Presenting with Chest Pain and Dyspnea

## Introduction
A patient presenting with chest pain and dyspnea warrants a thorough evaluation to determine the underlying cause. These symptoms are highly suggestive of ischemic heart disease and renovascular hypertension, conditions that require prompt diagnosis and management to prevent adverse outcomes.

## Clinical Presentation
The patient exhibits symptoms of **chest pain** and **dyspnea**, which are classic indicators of cardiac distress. The chest pain is described as dull and aching, occurring with exertion and at rest, which raises the concern for unstable angina. Dyspnea may correlate with physical findings such as a third heart sound and pulmonary crackles, suggesting left ventricular dysfunction and potential congestive heart failure due to myocardial ischemia.

## Risk Factors
Several risk factors contribute to the patient's presentation:

- **Hypertension**: A longstanding history increases the risk of coronary artery disease.
- **Peptic Ulcer Disease**: May complicate the clinical picture and influence medication choices.
- **Surgical Menopause**: Hormonal changes can impact cardiovascular health.
- **Family History of Early Atherosclerotic Cardiovascular Disease**: Genetics play a significant role in the development of coronary artery disease.

## Diagnostic Approach
A comprehensive diagnostic plan is essential:

1. **Telemetry Monitoring**: Admitting the patient to telemetry allows for continuous cardiac monitoring to detect arrhythmias or ischemic changes.
2. **Pharmacologic Interventions**:
   - **Diuretics**: Initiate to manage volume overload and reduce high blood pressure.
   - **Platelet Inhibitors (e.g., Aspirin)**: Decrease the risk of myocardial infarction by preventing platelet aggregation.
   - **Nitrates**: Alleviate chest pain and reduce the risk of arterial occlusion while also helping to lower blood pressure.
   - **Analgesics (e.g., Morphine)**: Consider for prolonged chest pain unresponsive to nitrates.
3. **Ventriculogram with Catheterization**: Assess cardiac size and identify any wall motion abnormalities indicative of ischemic heart disease.
4. **Orthostatic Blood Pressure Measurements**: Evaluate for autonomic dysfunction or volume depletion as part of vital sign monitoring.

## Physical Examination Findings
- **Cardiovascular**:
  - Presence of a systolic murmur, particularly a grade 2/6 systolic decrescendo murmur heard best at the second right intercostal space radiating to the neck, raises the possibility of aortic stenosis. However, a normal carotid upstroke may indicate that the murmur is not clinically significant.
  - Third heart sound detected at the apex suggests left ventricular dysfunction.
- **Respiratory**:
  - Clear lung fields except for crackles at the bases, consistent with pulmonary congestion.
- **Neurological**:
  - Absence of motor and sensory deficits with normal reflexes and gait indicates no apparent neurological compromise.
- **Breast Examination**:
  - Cystic changes bilaterally without masses or nipple discharge suggest fibrocystic breast disease.

## Long-Term Management
Effective long-term care strategies are crucial for preventing recurrence and managing underlying conditions:

- **Cholesterol Monitoring**: Regular assessment of lipid profiles to manage and mitigate the risk of atherosclerosis.
- **Lifestyle Modifications**:
  - **Exercise Program**: Structured physical activity to improve cardiovascular health and aid in weight loss.
  - **Low-Fat Diet**: Dietary adjustments to reduce cholesterol levels and manage hypertension.
- **Pharmacotherapy**:
  - **Cholesterol-Lowering Medications (e.g., HMG-CoA Reductase Inhibitors)**: Prescribe if lipid levels remain elevated despite lifestyle changes.

## Additional Considerations
- **Fibrocystic Breast Disease Evaluation**: Monitor breast health as part of comprehensive care.
- **Aortic Stenosis Assessment**: Further investigate the significance of the detected murmur to rule out valvular heart disease.

## Conclusion
This patient's presentation of chest pain and dyspnea, combined with significant risk factors and diagnostic findings, points towards ischemic heart disease as the most likely diagnosis. A structured approach encompassing immediate pharmacologic interventions, detailed diagnostic evaluations, and comprehensive long-term management plans is essential to optimize patient outcomes and prevent complications.